<a href="https://colab.research.google.com/github/JulesBendo/Test/blob/main/TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import dask.dataframe as dk
import requests
import zipfile
import os

In [3]:
# URL du fichier zip
zip_url = 'http://rhobs-public.s3-website.eu-west-3.amazonaws.com/data.zip'

In [4]:
# Téléchargement du fichier zip
response = requests.get(zip_url)
zip_file = 'data.zip'

In [5]:
# Ouvrir un fichier et écrire le contenu téléchargé dans ce fichier
with open(zip_file, 'wb') as f:
    f.write(response.content)

In [6]:
# Extraire le contenu du fichier zip dans le répertoire
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()

In [7]:
#Etant donné que le fichier se nomme "-" on le renomme en "data.csv"
os.rename('-', 'data.csv')

In [8]:
#on supprime ce zip à présent
os.remove(zip_file)

In [9]:
!head data.csv
#Parfait, à présent prenons connaissance des objectifs du test.

Your mission shall you accept it is to
* Explain every column.
* Find the 3 richest person.

This is a simple test, it should not take you more 30minutes to code.



a,b,c,d,e,f,g
0,longue description 0,Victor,Évrat,1779,0,1779


In [10]:
#Créons notre DataFrame en sélectionnant les lignes contenants les données.
df = dk.read_csv('data.csv', delimiter=',', header=None, skiprows=9,
                 names=['a', 'b', 'c', 'd', 'e', 'f', 'g'])

In [11]:
df.head(5)

,a,b,c,d,e,f,g
0,0,longue description 0,Victor,Évrat,1779,0,1779
1,1,longue description 1,Ugo,Ogier,3218,0,3218
2,2,longue description 2,Gerard,Zola,3909,0,3909
3,3,longue description 3,Ingrid,Quesada,2672,0,2672
4,4,longue description 4,Xavier,Yvars,4224,0,4224


In [12]:
nb_lignes = len(df)
print("Le DataFrame contient {} lignes.".format(nb_lignes))

Le DataFrame contient 300000000 lignes.


In [13]:
#Renomons les colonnes évidentes pour une meilleure visibilité.
df = df.rename(columns={'a': 'id', 'b': 'description', 'c': 'first_name', 'd': 'last_name', 'e': 'column1', 'f': 'column2', 'g': 'column3'})

In [14]:
df.head(100)

,id,description,first_name,last_name,column1,column2,column3
0,0,longue description 0,Victor,Évrat,1779,0,1779
1,1,longue description 1,Ugo,Ogier,3218,0,3218
2,2,longue description 2,Gerard,Zola,3909,0,3909
3,3,longue description 3,Ingrid,Quesada,2672,0,2672
4,4,longue description 4,Xavier,Yvars,4224,0,4224
...,...,...,...,...,...,...,...
95,95,longue description 7,Salomé,Walliand,2277,0,2277
96,96,longue description 0,Karim,Yvars,2749,0,2749
97,97,longue description 1,Zoé,Dupond,3534,0,3534
98,98,longue description 2,Régis,Kléber,3677,0,3677


In [15]:
df.tail(100)

,id,description,first_name,last_name,column1,column2,column3
1019267,299999900,longue description 4,Karim,Quesada,676,0,889761323
1019268,299999901,longue description 5,Ugo,Lombard,4597,0,889184263
1019269,299999902,longue description 6,Gerard,Lombard,0,-385,890200848
1019270,299999903,longue description 7,Ingrid,Parmentier,3381,0,888151547
1019271,299999904,longue description 0,Céline,Yvars,0,-743,885051641
...,...,...,...,...,...,...,...
1019362,299999995,longue description 3,Yann,Urbain,980,0,886572089
1019363,299999996,longue description 4,Océane,Geneau,3664,0,889929743
1019364,299999997,longue description 5,William,Niels,3341,0,887882801
1019365,299999998,longue description 6,Salomé,Truchot,0,-685,886712436


In [16]:
description = df.describe().compute()
print(description)

                 id       column1       column2       column3
count  3.000000e+08  3.000000e+08  3.000000e+08  3.000000e+08
mean   1.500000e+08  2.083416e+03 -8.340646e+01  4.437913e+08
std    8.660254e+07  1.613909e+03  2.206263e+02  2.562233e+08
min    0.000000e+00  0.000000e+00 -1.000000e+03 -2.470000e+03
25%    7.484619e+07  5.070000e+02  0.000000e+00  2.223327e+08
50%    1.499099e+08  2.010000e+03  0.000000e+00  4.447691e+08
75%    2.248275e+08  3.508000e+03  0.000000e+00  6.664433e+08
max    3.000000e+08  5.000000e+03  0.000000e+00  8.937760e+08


In [17]:
def top_3(df):
    numeric_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'id']
    for col in numeric_cols:
      Trois_max, Trois_min = df[col].nlargest(3), df[col].nsmallest(3)
      print(f"Trois plus petites valeurs de la colonne {col} :")
      print(Trois_min.compute())
      print(f"\nTrois plus grandes valeurs de la colonne {col} :")
      print(Trois_max.compute())


In [18]:
top_3(df)

Trois plus petites valeurs de la colonne column1 :
6     0
12    0
17    0
Name: column1, dtype: int64

Trois plus grandes valeurs de la colonne column1 :
1979    5000
2910    5000
8284    5000
Name: column1, dtype: int64
Trois plus petites valeurs de la colonne column2 :
3401    -1000
12145   -1000
12503   -1000
Name: column2, dtype: int64

Trois plus grandes valeurs de la colonne column2 :
0    0
1    0
2    0
Name: column2, dtype: int64
Trois plus petites valeurs de la colonne column3 :
2552   -2470
4573   -2126
2979   -1880
Name: column3, dtype: int64

Trois plus grandes valeurs de la colonne column3 :
1017731    893775958
1018905    893775893
1017069    893775523
Name: column3, dtype: int64


In [23]:
#Maintenant que les données sont plus claires renomons les autres colonnes.
df = df.rename(columns={'column1': 'deposit', 'column2': 'withdrawal', 'column3': 'balance'})

In [24]:
df

,id,description,first_name,last_name,deposit,withdrawal,balance
npartitions=292,,,,,,,
,int64,object,object,object,int64,int64,int64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


**MISSION 1**:

*   id(a)

    Cette colonne semble contenir des identifiants uniques pour chaque entrée dans nos données.

*   description(b)

    Cette colonne pourrait contenir des descriptions ou des informations textuelles décrivant chaque entrée dans vos données. On remarque qu'elles sont simplement notées "longue description 0-7".


*   first_name(c)

    Cette colonne semble contenir les prénoms des personnes associées à chaque entrée dans les données.

*   last_name(d)

    Cette colonne semble contenir les noms de famille des personnes  associées à chaque entrée dans les données.

*   deposit(e)

    Cette colonne présente des valeurs maximales de 5000 et des valeurs minimales de 0.
    Ceci pourrait laisser penser aux dépôts (les dépôts étant en général des ajouts de fonds) avec 5000 comme plafond et 0 comme la valeur théorique minimale du dépôt (dans le cas d'un client n'effectuant aucun dépôt).

*   withdrawal(f)

    Cette colonne présente des valeurs maximales de 0 et des valeurs minimales de -1000.
    Ceci pourrait laisser penser aux retraits(les retraits étant généralement des déductions de fonds) avec 1000 comme plafond et 0 comme la valeur théorique minimale du retrait (dans le cas d'un client n'effectuant aucun retrait).

*   balance(g)

    Cette colonne présente une valeur maximale de 893775958 et une valeur minimale de -2470.
    En analogie avec ce qui précède, ceci semble être le solde courant du compte, positif lorsque le client est en règle et négatif lorsqu'il est endetté.

In [21]:
id = [1017731, 1018905, 1017069]

# Sélectionnons les lignes correspondant aux id
selection = df[df['id'].isin(id)]

Personne = selection[['first_name', 'last_name']]

print(Personne.head())

        first_name last_name
1017069    William     Ferry
1017731   Amandine     Ferry
1018905     Céline    Ivanov


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:8110: UserWarning: Insufficient elements for `head`. 5 elements requested, only 3 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


**MISSION 2**
 Les trois personnes les plus riches sont respectivement :


1.    William Ferry

     893775958

2.   Amandine Ferry

     893775893

1.   Céline Ivanov
   
     893775523



